# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

c:\Users\Zhuravel\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
device = '/device:gpu:0'

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170498071/170498071 [==============================] - 41s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [5]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()


(64, 10)


c:\Users\Zhuravel\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [11]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.flatten = tf.keras.layers.Flatten()

        self.conv1 = tf.keras.layers.Conv2D(channel_1, [5,5], [1,1], padding='same',
                                            kernel_initializer=initializer,
                                            activation='relu')

        self.conv2 = tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='same',
                                            kernel_initializer=initializer,
                                            activation='relu')

        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [12]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [13]:
print_every = 100

def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [14]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.370253562927246, Accuracy: 9.375, Val Loss: 3.0852506160736084, Val Accuracy: 10.699999809265137
Iteration 100, Epoch 1, Loss: 2.2300503253936768, Accuracy: 28.728342056274414, Val Loss: 1.8765554428100586, Val Accuracy: 37.5
Iteration 200, Epoch 1, Loss: 2.07775616645813, Accuracy: 32.245025634765625, Val Loss: 1.8212316036224365, Val Accuracy: 40.400001525878906
Iteration 300, Epoch 1, Loss: 2.001938819885254, Accuracy: 34.02720260620117, Val Loss: 1.8420242071151733, Val Accuracy: 38.400001525878906
Iteration 400, Epoch 1, Loss: 1.9341404438018799, Accuracy: 35.76215744018555, Val Loss: 1.6765720844268799, Val Accuracy: 43.70000076293945
Iteration 500, Epoch 1, Loss: 1.8883817195892334, Accuracy: 36.80451583862305, Val Loss: 1.6162469387054443, Val Accuracy: 45.20000076293945
Iteration 600, Epoch 1, Loss: 1.8566640615463257, Accuracy: 37.70278549194336, Val Loss: 1.622523307800293, Val Accuracy: 44.29999923706055
Iteration 700, Epoch 1, Loss: 1.83075726

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [15]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, 0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.3733906745910645, Accuracy: 9.375, Val Loss: 5.220108985900879, Val Accuracy: 7.90000057220459
Iteration 100, Epoch 1, Loss: 2.038524866104126, Accuracy: 28.991334915161133, Val Loss: 1.7426950931549072, Val Accuracy: 37.20000076293945
Iteration 200, Epoch 1, Loss: 1.8400808572769165, Accuracy: 35.370025634765625, Val Loss: 1.5188322067260742, Val Accuracy: 48.10000228881836
Iteration 300, Epoch 1, Loss: 1.7342078685760498, Accuracy: 38.99501419067383, Val Loss: 1.4446882009506226, Val Accuracy: 50.5
Iteration 400, Epoch 1, Loss: 1.648870587348938, Accuracy: 42.01605224609375, Val Loss: 1.384893536567688, Val Accuracy: 52.39999771118164
Iteration 500, Epoch 1, Loss: 1.5895154476165771, Accuracy: 44.01509475708008, Val Loss: 1.3554160594940186, Val Accuracy: 52.999996185302734
Iteration 600, Epoch 1, Loss: 1.5505696535110474, Accuracy: 45.43989181518555, Val Loss: 1.2981321811676025, Val Accuracy: 56.5
Iteration 700, Epoch 1, Loss: 1.515438199043274, Accura

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [16]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.7810544967651367, Accuracy: 20.3125, Val Loss: 2.6864430904388428, Val Accuracy: 15.199999809265137
Iteration 100, Epoch 1, Loss: 2.229069232940674, Accuracy: 29.11509895324707, Val Loss: 1.8701202869415283, Val Accuracy: 36.0
Iteration 200, Epoch 1, Loss: 2.0735161304473877, Accuracy: 32.5171012878418, Val Loss: 1.8510905504226685, Val Accuracy: 36.89999771118164
Iteration 300, Epoch 1, Loss: 1.9947561025619507, Accuracy: 34.281558990478516, Val Loss: 1.848604440689087, Val Accuracy: 39.0
Iteration 400, Epoch 1, Loss: 1.9257702827453613, Accuracy: 36.06998062133789, Val Loss: 1.7339420318603516, Val Accuracy: 40.79999923706055
Iteration 500, Epoch 1, Loss: 1.8826760053634644, Accuracy: 37.21931457519531, Val Loss: 1.6776422262191772, Val Accuracy: 43.29999923706055
Iteration 600, Epoch 1, Loss: 1.8525437116622925, Accuracy: 38.07976150512695, Val Loss: 1.6818976402282715, Val Accuracy: 42.0
Iteration 700, Epoch 1, Loss: 1.8271900415420532, Accuracy: 38.76

Альтернативный менее гибкий способ обучения:

In [17]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)



313/313 [==============================] - 3s 11ms/step - loss: 1.6514 - sparse_categorical_accuracy: 0.4386


[1.6513885259628296, 0.43860000371932983]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [18]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (28,28,1)
    channel_1, channel_2, num_classes = 28, 14, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Conv2D(channel_1, [5,5], [1,1], padding='same',
                               kernel_initializer=initializer,
                               activation='relu'),
        tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='same',
                               kernel_initializer=initializer,
                               activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes,
                              activation='softmax',
                              kernel_initializer=initializer)
    ]
    model = tf.keras.Sequential(layers)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0063610076904297, Accuracy: 3.125, Val Loss: 2.638956069946289, Val Accuracy: 9.899999618530273
Iteration 100, Epoch 1, Loss: 2.044039726257324, Accuracy: 26.547029495239258, Val Loss: 1.8566898107528687, Val Accuracy: 33.5
Iteration 200, Epoch 1, Loss: 1.9263343811035156, Accuracy: 31.4443416595459, Val Loss: 1.7317354679107666, Val Accuracy: 38.80000305175781
Iteration 300, Epoch 1, Loss: 1.853524088859558, Accuracy: 34.29713439941406, Val Loss: 1.680908441543579, Val Accuracy: 39.79999923706055
Iteration 400, Epoch 1, Loss: 1.7920745611190796, Accuracy: 36.67783737182617, Val Loss: 1.6088413000106812, Val Accuracy: 44.29999923706055
Iteration 500, Epoch 1, Loss: 1.7452157735824585, Accuracy: 38.41691589355469, Val Loss: 1.5444978475570679, Val Accuracy: 45.10000228881836
Iteration 600, Epoch 1, Loss: 1.7131439447402954, Accuracy: 39.535667419433594, Val Loss: 1.5150470733642578, Val Accuracy: 47.099998474121094
Iteration 700, Epoch 1, Loss: 1.6830477714

In [19]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)


313/313 [==============================] - 3s 8ms/step - loss: 1.4227 - sparse_categorical_accuracy: 0.4868


[1.4227068424224854, 0.4867999851703644]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [20]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [21]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.9117345809936523, Accuracy: 7.8125, Val Loss: 2.824190139770508, Val Accuracy: 12.399999618530273
Iteration 100, Epoch 1, Loss: 2.2366700172424316, Accuracy: 28.233293533325195, Val Loss: 1.9646798372268677, Val Accuracy: 36.39999771118164
Iteration 200, Epoch 1, Loss: 2.0720741748809814, Accuracy: 32.2527961730957, Val Loss: 1.897971272468567, Val Accuracy: 37.099998474121094
Iteration 300, Epoch 1, Loss: 1.998146414756775, Accuracy: 34.20888900756836, Val Loss: 1.92055082321167, Val Accuracy: 35.29999923706055
Iteration 400, Epoch 1, Loss: 1.9285513162612915, Accuracy: 36.02711868286133, Val Loss: 1.7719783782958984, Val Accuracy: 41.5
Iteration 500, Epoch 1, Loss: 1.8853892087936401, Accuracy: 37.0914421081543, Val Loss: 1.6971889734268188, Val Accuracy: 41.900001525878906
Iteration 600, Epoch 1, Loss: 1.8556714057922363, Accuracy: 37.87177658081055, Val Loss: 1.713507056236267, Val Accuracy: 43.099998474121094
Iteration 700, Epoch 1, Loss: 1.8297194242

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [23]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        channel_1, channel_2, num_classes = 28, 14, 10
        dp_rate = 0.2
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, [3,3], [1,1], padding='same',
                                  kernel_initializer=initializer)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()
        self.dp1 = tf.keras.layers.Dropout(rate=dp_rate)
        self.conv2 = tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='same',
                                  kernel_initializer=initializer)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()
        self.dp2 = tf.keras.layers.Dropout(rate=dp_rate)
        self.fl = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes,
                                  activation='softmax',
                                  kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = input_tensor
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dp1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dp2(x)

        x = self.fl(x)
        x = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.8542490005493164, Accuracy: 14.0625, Val Loss: 5.335721969604492, Val Accuracy: 10.800000190734863
Iteration 700, Epoch 1, Loss: 1.6460920572280884, Accuracy: 45.198822021484375, Val Loss: 1.2615634202957153, Val Accuracy: 55.900001525878906
Iteration 1400, Epoch 2, Loss: 1.1793444156646729, Accuracy: 58.742618560791016, Val Loss: 1.176149845123291, Val Accuracy: 58.60000228881836
Iteration 2100, Epoch 3, Loss: 1.0282484292984009, Accuracy: 64.04602813720703, Val Loss: 1.0620017051696777, Val Accuracy: 63.099998474121094
Iteration 2800, Epoch 4, Loss: 0.9474617838859558, Accuracy: 67.18128967285156, Val Loss: 1.05313241481781, Val Accuracy: 63.70000076293945
Iteration 3500, Epoch 5, Loss: 0.8915284872055054, Accuracy: 69.03246307373047, Val Loss: 1.029054880142212, Val Accuracy: 63.599998474121094
Iteration 4200, Epoch 6, Loss: 0.8531863689422607, Accuracy: 70.3714599609375, Val Loss: 1.0187292098999023, Val Accuracy: 63.900001525878906
Iteration 4900, Epo

Опишите все эксперименты, результаты. Сделайте выводы.

В данной работе была реализована сверточная нейронная сеть с двумя сверточными слоями и полносвязным слоем. Архитектура нейронной сети включает в себя следующие слои:

- Каждый сверточный слой использует активацию ReLU. После каждого сверточного слоя применяется пакетная нормализация и слой Dropout с коэффициентом отсечения 0.2.

- После первого сверточного слоя идет второй сверточный слой с аналогичными параметрами.

- Полносвязный слой: После сверточных слоев данные проходят через слой Flatten для преобразования в одномерный вектор, затем данные передаются в полносвязный слой с функцией активации softmax для классификации на выходе.